In [14]:
from datetime import datetime

import eikon as ek
import pandas as pd

from utilities import columns, filter_dataset

# CONSTANTS

In [2]:
with open('key.secret') as file:
    KEY = file.read()

In [3]:
FILENAME = 'db.xlsx'

In [4]:
try:
    ek.set_app_key(KEY)
    bonds, err = ek.get_data(['CN151819SH='], 'TR.FirstAnnounceDate')
    # sanity check to prevent silent error
    sanity_check = bonds['First Announcement Date'][0]
    if sanity_check != '2019-07-11':
        raise ValueError(f'Sanity check not passed: {sanity_check}')
except ek.EikonError:
    raise ValueError('Please check key.secret, then run Eikon Desktop or Eikon API Proxy')

# CHECK DB

In [5]:
green_bonds = pd.read_excel(FILENAME, parse_dates=['Issue Date'])

In [6]:
col_set = set(green_bonds.columns)

if columns.col_names != col_set:
    if len(col_set) < len(columns.col_names):
        raise TypeError(
            f"{columns.col_names - set(green_bonds.columns)} ARE MISSING COLUMNS!")

    elif len(col_set) > len(columns.col_names):
        raise TypeError(f"{set(green_bonds.columns) - columns.col_names} ARE ADDITIONAL COLUMNS!\nPlease ADD THEM"
                        " to columns.py OR REMOVE THEM")

    else:
        raise TypeError(
            f"{columns.col_names - set(green_bonds.columns)} ARE MISSING COLUMNS!")

# FILTER DB

In [7]:
green_bonds = filter_dataset.filter_dataset(green_bonds)

Excluded 459 rows over 2323. Green bonds before 2012 must be due to errors and 2020 might pollute data due to COVID and year not finished yet.


# GET ADDITIONAL COLUMNS

In [11]:
ADDITIONAL_COLUMNS = ['TR.FirstAnnounceDate', 'TR.IssuerRating',
                      'TR.IsPublic', 'TR.FiParentLongName',
                      'TR.UltimateParentID', 'TR.FiCouponClassDescription']

In [12]:
# Moodys Rating has WR for non-active bonds, so only Issuer Rating is relevant
bonds, err = ek.get_data(green_bonds.ISIN.dropna().to_list(),
                         ADDITIONAL_COLUMNS)

In [13]:
bonds

,Instrument,First Announcement Date,Issuer Rating,Organization Is Public Flag,Parent Long Name,Ultimate Parent Id,Coupon Class Description
0,XS1890845875,2019-02-05,A-,False,IBERDROLA SA,4295889602,Floating Coupon
1,CND10000C3L5,2016-09-01,Ba1,True,XINJIANG GOLDWIND SCIENCE & TECHNOLOGY CO LTD,4295864350,Floating Coupon
2,XS2082433736,2019-11-26,,False,AC ENERGY FINANCE INTERNATIONAL LTD,5067501160,Fixed Coupon
3,CND100017CJ8,2017-09-08,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Floating Coupon
4,CND10001WQ91,2019-01-02,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Floating Coupon
...,...,...,...,...,...,...,...
1842,XS1207105161,2015-03-20,B,False,PAPREC HOLDING SA,5045515418,Fixed Coupon
1843,US83417KFC71,2015-11-30,,False,TESLA INC,4297089638,Fixed Coupon
1844,XS1512929842,2016-10-28,P-1,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
1845,XS1566937154,2017-07-26,F1+,False,RUE LA BOETIE SAS,4298458216,Fixed Coupon


In [11]:
isin_merged = green_bonds.merge(right=bonds, how='outer', left_on='ISIN', right_on='Instrument')
isin_merged.drop(columns=['Instrument'])

,Issuer,Ticker,Coupon,Maturity,Issue Date,ISIN,Preferred RIC,Principal Currency,Country of Issue,Issuer Type,...,SIC,Use of Proceeds,PBOC Bond Rating,Current Coupon Class,First Announcement Date,Issuer Rating,Organization Is Public Flag,Parent Long Name,Ultimate Parent Id,Coupon Class Description
0,Iberdrola International BV,IBERD,3.25,NaN,2019-02-12,XS1890845875,ES189084587=,Euro,Eurobond Market,Corporate,...,6726,Energy Efficiency,NaN,Fixed Coupon,2019-02-05,A-,False,IBERDROLA SA,4295889602,Floating Coupon
1,Xinjiang Goldwind Science & Technology Co Ltd,XINGO,4.2,NaN,2016-09-01,CND10000C3L5,CN131656002=,Chinese Yuan,China (Mainland),Corporate,...,3511,Alternative Energy,AAA/1,Fixed Coupon,2016-09-01,Ba1,True,XINJIANG GOLDWIND SCIENCE & TECHNOLOGY CO LTD,4295864350,Floating Coupon
2,Ac Energy Finance International Ltd,ACFTE,5.65,NaN,2019-12-04,XS2082433736,KY208243373=,US Dollar,Eurobond Market,Corporate,...,6799,Energy Efficiency,NaN,Fixed Coupon,2019-11-26,,False,AC ENERGY FINANCE INTERNATIONAL LTD,5067501160,Fixed Coupon
3,CGN Wind Energy Ltd,SASABZ,5.25,NaN,2017-09-14,CND100017CJ8,CN131751002=,Chinese Yuan,China (Mainland),Corporate,...,4911,Eligible Green Projects,AAA/1,Fixed Coupon,2017-09-08,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Floating Coupon
4,CGN Wind Energy Ltd,SASABZ,4.31,NaN,2019-01-07,CND10001WQ91,CN131900001=,Chinese Yuan,China (Mainland),Corporate,...,4911,Electric & Public Power,AAA/1,Fixed Coupon,2019-01-02,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Floating Coupon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,Paprec Holding SA,PAPIG,7.375,2023-04-01 00:00:00,2015-03-27,XS1207105161,NaN,Euro,Eurobond Market,Corporate,...,6799,Repay Intercompany Debt,NaN,Fixed Coupon,2015-03-20,B,False,PAPREC HOLDING SA,5045515418,Fixed Coupon
1860,Tesla Energy Operations Inc,TSLAS,1.6,2016-12-17 00:00:00,2015-12-17,US83417KFC71,NaN,US Dollar,United States,Corporate,...,4911,Alternative Energy,NaN,Fixed Coupon,2015-11-30,,False,TESLA INC,4297089638,Fixed Coupon
1861,Bank of China (London Branch),BKCHNL,1.875,2019-11-09 00:00:00,2016-11-09,XS1512929842,NaN,US Dollar,Eurobond Market,Corporate,...,6081,Clean Transport,NaN,Fixed Coupon,2016-10-28,P-1,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
1862,Credit Agricole Corporate and Investment Bank SA,CAGRAB,9.55,2020-07-27 00:00:00,2017-07-26,XS1566937154,NaN,Turkish Lira,Eurobond Market,Corporate,...,6029,Alternative Energy,NaN,Fixed Coupon,2017-07-26,F1+,False,RUE LA BOETIE SAS,4298458216,Fixed Coupon


In [12]:
# get RICs from where we don't have ISIN
rics = green_bonds[green_bonds.ISIN.isna()]['Preferred RIC'].dropna().to_list()

In [13]:
bonds, err = ek.get_data(rics,
                         ADDITIONAL_COLUMNS)

In [14]:
bonds

,Instrument,First Announcement Date,Issuer Rating,Organization Is Public Flag,Parent Long Name,Ultimate Parent Id,Coupon Class Description
0,CN114265SZ=,2017-12-01,AA+,False,SUZHOU GCL NEW ENERGY INVESTMENT CO LTD,4295871879,Fixed Coupon
1,CN143952SH=,2018-03-07,AAA,True,"HEBEI, PROVINCE OF",5000040363,Fixed Coupon
2,CN150502SH=,2018-06-21,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
3,CN150701SH=,2018-11-07,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
4,CN155057SH=,2018-11-29,AAA,True,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
5,CN155956SH=,2019-02-27,AAA,True,"HEBEI, PROVINCE OF",5000040363,Fixed Coupon
6,CN143518SH=,2018-03-14,AAA,False,"SHANGHAI, MUNICIPALITY OF",5000063159,Variable Coupon
7,CN143745SH=,2018-07-31,AAA,False,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
8,CN143525SH=,2018-08-15,AAA,True,"CHINA, PEOPLE'S REPUBLIC OF (GOVERNMENT)",5000036491,Fixed Coupon
9,CN150646SH=,2018-08-21,AA,False,"URUMQI, CITY OF",5050923137,Fixed Coupon


In [15]:
isin_merged.set_index('Preferred RIC', inplace=True)
bonds.set_index('Instrument', inplace=True)

In [16]:
# updating entire df gives error
for col in bonds.columns:
    isin_merged[col].fillna(bonds[col], inplace=True)

In [17]:
isin_merged.reset_index(inplace=True)

In [18]:
bonds, err = ek.get_data(list(isin_merged['Ultimate Parent Id'].dropna().unique().astype(str)),
                         ['TR.IsPublic'])

In [19]:
bonds.rename(columns={'Organization Is Public Flag': 'IsParentPublic'}, inplace=True)
bonds

,Instrument,IsParentPublic
0,4295889602,True
1,4295864350,True
2,5067501160,False
3,5000036491,False
4,4295866323,True
...,...,...
501,4296014747,False
502,4296062372,True
503,4296698857,False
504,4295891741,True


In [20]:
isin_merged['IsParentPublic'] = isin_merged['Ultimate Parent Id'].map(
    dict(zip(bonds.Instrument, bonds.IsParentPublic)))

In [21]:
isin_merged.rename({'Organization Is Public Flag': 'IsPublic'}, inplace=True)

In [22]:
isin_merged.to_csv('cleaned_green_bonds.csv', index=False)

# GET FINANCIAL DATA

In [84]:
CURRENT_YEAR = datetime.now().year
years_to_2012 = CURRENT_YEAR - 2012

assets, errors = ek.get_data(list(isin_merged.ISIN.dropna().unique()),
                             [
    f"TR.F.TotAssets(SDate=0,EDate=-{years_to_2012},Period=FY0,Frq=FY)",
    f'TR.ROAActValue(SDate=0FY,EDate=-{years_to_2012}FY,Period=FY0,Frq=FY)'
]
)

In [85]:
assets

,Instrument,Total Assets,Return On Assets - Actual
0,XS1890845875,NaN,NaN
1,CND10000C3L5,1.030571e+11,2.397
2,CND10000C3L5,8.136405e+10,4.173
3,CND10000C3L5,7.278784e+10,4.452
4,CND10000C3L5,6.443716e+10,5.133
...,...,...,...
8354,US83417KBK34,1.342300e+09,NaN
8355,US83417KBK34,8.131730e+08,NaN
8356,US83417KBK34,3.712640e+08,NaN
8357,US83417KBK34,1.641540e+08,NaN


In [28]:
grouped_assets = assets.dropna().groupby('Instrument')

In [39]:
assets_dict = dict()

for name, asset_table in grouped_assets:
    company_assets = asset_table.sort_index().reset_index()
    company_assets['Year'] = [CURRENT_YEAR - x for x in company_assets.index]
    assets_dict[name] = dict(zip(company_assets.Year, company_assets['Total Assets']))

In [71]:
test, errors = ek.get_data('FR0013310505=',
                           [f'TR.NetProfitActValue(SDate=0FY,EDate=-{years_to_2012}FY,Period=FY0,Frq=FY)'])

In [72]:
test['Net Income - Actual']

0    2683000000
1    2425000000
2    2662000000
3    2477000000
4    2588000000
5    3027000000
6    5234000000
7    3800000000
8    3546218000
Name: Net Income - Actual, dtype: Int64

In [74]:
[net_income / assets for net_income, assets in (zip(test['Net Income - Actual'], assets_dict['FR0013310505'].values()))]

[0.016790472674022016,
 0.015777283314465654,
 0.017730118556014387,
 0.015627858850844483,
 0.01610875275429795,
 0.01831160581954569,
 0.033565913346843494,
 0.018819893618075022,
 0.016616925167517924]

In [82]:
test, errors = ek.get_data('FR0013310505=',
                           f'TR.ROAActValue(SDate=0FY,EDate=-{years_to_2012}FY,Period=FY0,Frq=FY)')

In [83]:
test

,Instrument,Return On Assets - Actual
0,FR0013310505=,1.710
1,FR0013310505=,1.600
2,FR0013310505=,1.540
3,FR0013310505=,2.069
4,FR0013310505=,1.588
5,FR0013310505=,1.880
6,FR0013310505=,2.867
7,FR0013310505=,1.814
8,FR0013310505=,1.780
